## Week 2 Lab exercises

In this week we will use python to explore strange new worlds, to seek out new life and new civilisation, and to boldly code what no one has coded before. 

The goal is to familiarise yourself with using functions, loops, arrays and plotting in Python, and to illustrate some of the software carpentry points that we discussed in the lecture.

In [1]:
# first we import the libraries we need
import numpy as np
import matplotlib.pyplot as plt
import time

# The following lines makes the plots appear in another window - if using AppsAnywhere they may appear
# in another window behind the notebook, and you may have to rearrange the desktop to see them.
%matplotlib

Using matplotlib backend: <object object at 0x000001D6FB45C240>


### ACTIVITY 1: 
In the code below, change the line as marked (UPDATE ME) to get the asteroid data from the data file `asteroid_locations.dat` that is in the `/data` folder

In [2]:
# Set up the plotting

def plot_asteroid_field(ax) :
    plt.axis([0,10.5,-4,4])
    major_ticks = np.arange(0, 11, 1)
    ax.set_xticks(major_ticks)
    plt.xlabel("time")
    plt.ylabel("position")
    ax.set_aspect('equal')
    ax.grid(which='major', alpha=0.8)

    # Mark the asteroid positions onto the plot - read in the data from data/asteroid_locations.dat
    # HINT: the function you need is called loadtxt
    asteroid_locations= np.loadtxt("data/asteroid_locations.dat") # UPDATE ME!
    for location in asteroid_locations :
        circle=plt.Circle(location, 0.5 , color='r')
        ax.add_artist(circle)
    plt.pause(0.3)

# Test the plot
fig = plt.figure()
ax = plt.gcf().gca()
plot_asteroid_field(ax)

In [5]:
# set up a list to store the times and positions
list_of_times = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
list_of_positions = np.zeros_like(list_of_times)

# we want to track how much we have moved during the simulation
# so zero this
total_distance_moved = 0

# print off the initial values of the times and positions
print("The list of times at the start is ", list_of_times)
print("The list of positions at the start is ", list_of_positions)


The list of times at the start is  [ 0  1  2  3  4  5  6  7  8  9 10]
The list of positions at the start is  [0 0 0 0 0 0 0 0 0 0 0]
4


### ACTIVITY 2: 

Captain Katy has asked you to plot a flight path through the asteroid field. Just like in a retro video game, the ship moves one step forward in the x direction at every timestep, and you can only control its movement in the y direction to avoid the asteroids.

If you run the code below you will see that the path does not currently avoid the asteroids :-( Update it so that it does, and so that you end the flight back at position 0.

Also add code where needed to correctly track the total_distance_moved.

In [4]:
# First make tests for the rules 

# Rule 1: always moving forward in the x direction
# np.arrange generates a sequence of values from start to stop with a given step size 
# .all() enforces equality for all elements 

def test_x_movement_unmodified(a_list_of_times):
    time_size = np.size(a_list_of_times)
    assert (a_list_of_times == np.arrange(time_size).all()), 'It is illegal to change the movement in the x direction'

# Test of the Test:
# use broken_list_of_times = np.array([1, 1, 2, 3, 4, 5, 6, 7, 8, 9,10])
# test_x_movement_unmodified(broken_list_of_times)

# Rule 2: must move in whole steps in the y direction (steps must be integer values)
# dtype defines the type of the array

def test_y_movement_integer(a_list_of_y_positions):
    assert (a_list_of_y_positions.dtype == 'int64'), 'It is illegal to move in non integer steps'

# Test of the Test:
# use broken_list_of_y_positions = np.array([0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0])
# test_y_movement_integer(broken_list_of_y_positions)

# Rule 3 : end back at y position 0
# np.size returns the number of elements in an array
# therefore np.size - 1 will give the last element in the array

def test_final_position(a_list_of_y_positions):
    last_index = np.size(a_list_of_y_positions) - 1
    assert a_list_of_y_positions[last_index] == 0, 'You have to finish at y position 0'

# Test of the Test 
# use broken_final_position = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
# test_final_position(broken_final_position)

# Rule 4: Avoid the Asteroids!
# reload the asteroid locations

def test_avoiding_asteroids(a_list_of_times,a_list_of_y_positions):
    asteroid_locations= np.loadtxt("data/asteroid_locations.dat")

# test we are avoiding the asteroids at the integer locations
# location[0] returns the x coordinate in the array where as location[1] returns the y coordinate
# use pythag to work out the distance to the asteroid 

    for iloc, locations in enumerate(asteroid_locations):
        for ix, x_position in enumerate(a_list_of_times):
            distance_to_asteroid = np.sprt((location[0] - x_position**2.0)
                                          + location[1] - a_list_of_y_positions[ix]**2.0)
            message = 'You have hit an asteroid at ' + str(location)
            assert distance_to_asteroid > 0.5, message

# This is the harder part now - whether the line hits any of the circles
     for iloc, locations in enumerate(asteroid_locations):
        for ix, x_position in enumerate(a_list_of_times):
            if ix > 0: # skips first point 

                # find the equation of the line through the 
                # segment of the points ix and ix - 1
                # using y = mx + c
                # for readibility assign the variables with nice names
                # and make sure they are floats (includes decimals)

                x2 = float(a_list_of_times[ix])
                x1 = float(a_list_of_times[ix-1])
                y2 = float(a_list_of_y_positions[ix])
                y1 = float(a_list_of_y_positions[ix-1])
                x_asteroid = float(locations[0])
                y_asteroid = float(locations[1])
                m_line = (y2 - y1) / (x2 - x1)
                c_line = y1 - m_line * x1

                # The line that intersects perpendicularly 
                # Treat the special case of zero gradient
                # We can never have a vertical line as we always move forward
                if m_line == 0:
                    x_intersect = x_asteroid 
                    y_intersect = y2

                # Now for the more general case 
                else:
                    m_perpendicular = -1.0/ m_line 
                    c_perpendicular = y_asteroid - m_perpendicular * x_asteroid

                    # Now find the intersection point (x_intersect, y_intersect)
                    x_intersect = (c_line - c_perpendicular) / (m_perpendicular - m_line)
                    y_intersect = m_line * x_intersect + c_line 
                

    

    


# Plot again the asteroid field, so that we can plot the path on top of it
#plt.ion() # Make the plotting interactive so we can update the path as we fly
fig = plt.figure()
ax = plt.gcf().gca()
plot_asteroid_field(ax)

# go through the list of times, where index is the index within the 
# list, and time_now is the entry at that index
for index, time_now in enumerate(list_of_times) :
    
    # the list of things to do when at a certain time starts with an 'if'
    # start with time is zero, we start at position 0

    if time_now == 0 :
        list_of_positions[index] = 0
        total_distance_moved = 0
    
    # alternative option denoted by 'elif' (= else if)
    # ie, if the time is between 1 and 3
    elif (time_now > 0 and time_now < 2) :
        step_size = 1
        # moving y position by the step size
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)
        

    elif (time_now == 8):
        step_size = -1
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)

    # finally, give the default option for all other cases using 'else'
    else :
        # nothing happens at other times
        step_size = 0
        # moving y position by the step size
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        total_distance_moved += 1

    position_now = list_of_positions[index]
    #ax.plot(time_now, position_now, 'o')
    #plt.pause(0.3) # This pause just slows down the plotting so it looks nicer

print("The list of times at the end is ", list_of_times)
print("The list of positions at the end is ", list_of_positions)

ax.plot(list_of_times, list_of_positions, 'g-')


# Uncomment to save the figure
# plt.savefig("flight_path.png")

# Print out some information
print("I moved a total of ", total_distance_moved, " units")

IndentationError: expected an indented block (1524359114.py, line 33)

### ACTIVITY 3:

Now we are on an exploratory mission. Captian Katy wants you to plot a course for all of the blue (M or Minshara-class) planets on the map contained in `data/planet_locations.dat`, but not touching any of the other dangerous D-class pink planets. 

In [12]:
# Plot the planetary locations

def plot_planet_locations(ax) :
    plt.axis([0,15.5,-4,4])
    major_ticks = np.arange(0, 16, 1)
    ax.set_xticks(major_ticks)
    plt.xlabel("time")
    plt.ylabel("position")
    ax.set_aspect('equal')
    ax.grid(which='major', alpha=0.8)

    # Mark the planet positions onto the plot - read in the data from data/planet_locations.dat
    # HINT: the function you need is called loadtxt
    planet_locations= np.loadtxt("data/planet_locations.dat") # UPDATE ME!

    for index, location in enumerate(planet_locations) :
        is_minshara_class =  (location[2] == 1)
        coordinates = np.array([location[0],location[1]])
        if(is_minshara_class) :
            circle=plt.Circle(coordinates, 0.5, color='b')
            plt.gcf().gca().add_artist(circle)
        else : # other planets are dangerous D class planets
            circle=plt.Circle(coordinates, 0.5, color='m')
            plt.gcf().gca().add_artist(circle)  

# Test the plot
fig = plt.figure()
ax = plt.gcf().gca()
plot_planet_locations(ax)

Update the code below to make sure you visit them all (same rules of motion as above) and return to position 0 at the end. What is the shortest distance in which you can do this?

In [19]:
# Plot again the asteroid field, so that we can plot the path on top of it
plt.ion() # Make the plotting interactive so we can update the path as we fly
fig = plt.figure()
ax = plt.gcf().gca()
plot_planet_locations(ax)   

# set up a list to store the times and positions
final_time = 15
list_of_times = np.arange(final_time+1)
list_of_positions = np.zeros_like(list_of_times)

# we want to track how much we have moved during the simulation
# so zero this
total_distance_moved = 0

# print off the initial values of the times and positions
print("The list of times at the start is ", list_of_times)
print("The list of positions at the start is ", list_of_positions)

# go through the list of times, where index is the index within the 
# list, and time_now is the entry at that index
for index, time_now in enumerate(list_of_times) :
    
    # the list of things to do when at a certain time starts with an 'if'
    # start with time is zero, we start at position 0
    if time_now == 0 :
        list_of_positions[index] = 0
        total_distance_moved = 0
    
    # alternative option denoted by 'elif' (= else if)
    elif (time_now > 0 and time_now < 2) :
        step_size = -2
        # moving y position by the step size
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)
    elif time_now == 8:
        step_size = -1
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)

    elif time_now == 9:
        step_size = 1
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)
    
    elif time_now == 12:
        step_size = 2
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)

    elif time_now == 13:
        step_size = 2
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)
    
    elif time_now == 15:
        step_size = -2
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        if step_size == 0: 
            total_distance_moved += 1
        else:
            total_distance_moved += np.sqrt(1**2 + step_size**2)

    # finally, give the default option for all other cases using 'else'
    else :
        # nothing happens at other times
        step_size = 0
        list_of_positions[index] = list_of_positions[index - 1] + step_size
        total_distance_moved += 1

    position_now = list_of_positions[index]
    plt.plot(time_now, position_now, 'o')
    plt.pause(0.4)

print("The list of times at the end is ", list_of_times)
print("The list of positions at the end is ", list_of_positions)

plt.plot(list_of_times, list_of_positions, 'g-')
plt.show()

# Uncomment to save the figure
#plt.savefig("myplot2.png")

# Print out some information
print("I moved a total of ", total_distance_moved, " units")

The list of times at the start is  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
The list of positions at the start is  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
The list of times at the end is  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
The list of positions at the end is  [ 0 -2 -2 -2 -2 -2 -2 -2 -3 -2 -2 -2  0  2  2  0]
I moved a total of  20.77269903474535  units


### ACTIVITY 4:

List below in markdown format all the software carpentry features that you note in the above code. Which ones are not ilustrated? Suggest ways to improve the code.

1. There are comments to help the reader know what is happening...
2. UPDATE ME
3. ...
